In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp '/content/drive/MyDrive/Colab Notebooks/sem_3.ipynb' '/content/drive/MyDrive/Tutorial_Deep_Learning_YT/sem_3.ipynb'

In [3]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 9.5 MB/s eta 0:00:00


In [18]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras import utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tensorflow.keras.metrics import AUC
%matplotlib inline

In [7]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [11]:
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train / 255
X_test = X_test / 255
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

In [26]:
def build_model(hp):
  model = Sequential()
  activation_choice = hp.Choice('activation', values=['relu', 'tanh', 'sigmoid', 'elu', 'selu'])
  model.add(Input(shape=(X_train.shape[1],)))
  model.add(Dense(units=hp.Int('units_input',
                                 min_value=512,
                                 max_value=1024,
                                 step=32),
                  activation=activation_choice))
  model.add(Dense(10, activation='softmax'))
  model.compile(optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [34]:
tuner_1 = Hyperband(
  build_model,
  objective='val_accuracy',

  max_epochs=150,
  factor=3,
  directory='my_dir_second',  # Директория для сохранения логов
)


In [35]:
tuner_1.search_space_summary()

Search space summary
Default search space size: 3
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'rmsprop', 'SGD'], 'ordered': False}


In [36]:
tuner_1.search(X_train, y_train,
             batch_size=256,
             epochs=25,
             validation_split=0.2,
             verbose=1)

Trial 254 Complete [00h 01m 41s]
val_accuracy: 0.09833333641290665

Best val_accuracy So Far: 0.8211666941642761
Total elapsed time: 01h 03m 29s


In [38]:
tuner_1.results_summary()

Results summary
Results in my_dir_second/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0146 summary
Hyperparameters:
activation: selu
units_input: 960
optimizer: adam
tuner/epochs: 150
tuner/initial_epoch: 50
tuner/bracket: 4
tuner/round: 4
tuner/trial_id: 0144
Score: 0.8211666941642761

Trial 0249 summary
Hyperparameters:
activation: elu
units_input: 928
optimizer: adam
tuner/epochs: 150
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8199166655540466

Trial 0209 summary
Hyperparameters:
activation: tanh
units_input: 1024
optimizer: adam
tuner/epochs: 150
tuner/initial_epoch: 50
tuner/bracket: 3
tuner/round: 3
tuner/trial_id: 0204
Score: 0.8184166550636292

Trial 0246 summary
Hyperparameters:
activation: tanh
units_input: 768
optimizer: adam
tuner/epochs: 150
tuner/initial_epoch: 50
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0242
Score: 0.8175833225250244

Trial 0245 summary
Hyperparameters:
activation: elu
units_in

In [39]:
models = tuner_1.get_best_models(num_models=3)

/usr/local/lib/python3.10/dist-packages/keras_tuner/src/tuners/hyperband.py:435: UserWarning: Model 'sequential' had a build config, but the model cannot be built automatically in `build_from_config(config)`. You should implement `def build_from_config(self, config)`, and you might also want to implement the method  that generates the config at saving time, `def get_build_config(self)`. The method `build_from_config()` is meant to create the state of the model (i.e. its variables) upon deserialization.
  model.build_from_config(
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [41]:
for model in models:
  model.summary()
  model.evaluate(X_test, y_test)
  print()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 960)                 │         753,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           9,610 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 763,210 (2.91 MB)

 Trainable params: 763,210 (2.91 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 1024)                │         803,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │          10,250 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 814,090 (3.11 MB)

 Trainable params: 814,090 (3.11 MB)

 Non-trainable params: 0 (0.00 B)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8118 - loss: 0.5363

